In [1]:
'''
实验目的：实现基于CNN、RNN的文本分类

实验内容：
1）词嵌入初始化方式：随机embedding、加载glove
2）CNN/RNN的特征抽取
3）Dropout


参考：
https://arxiv.org/abs/1408.5882
https://github.com/yokusama/CNN_Sentence_Classification
https://torchtext.readthedocs.io/en/latest/
http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

'''

'\n实验目的：实现基于CNN、RNN的文本分类\n\n实验内容：\n1）词嵌入初始化方式：随机embedding、加载glove\n2）CNN/RNN的特征抽取\n3）Dropout\n\n\n参考：\nhttps://arxiv.org/abs/1408.5882\nhttps://github.com/yokusama/CNN_Sentence_Classification\nhttps://torchtext.readthedocs.io/en/latest/\nhttp://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/\n\n'

In [2]:
import os
import time
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import sklearn
print(os.getcwd())

dir_all_data='data\\task2_all_data.tsv'

BATCH_SIZE=10

cpu=True   #True   False 
if cpu :
    USE_CUDA = False
    DEVICE = torch.device('cpu')
else:
    USE_CUDA = torch.cuda.is_available()
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(0)


D:\workspace\nlp_beginer_solution


In [3]:
#从文件中读取数据
data_all=pd.read_csv(dir_all_data,sep='\t')
#print(all_data.shape)    #(156060, 4)
#print(all_data.keys())   #['PhraseId', 'SentenceId', 'Phrase', 'Sentiment']
idx =np.arange(data_all.shape[0])
#print(data_all.head())
#print(type(idx))   #<class 'numpy.ndarray'>

In [4]:
#shuffle，划分验证集、测试集,并保存
seed=0
np.random.seed(seed)
#print(idx)
np.random.shuffle(idx)  
#print(idx)

train_size=int(len(idx) * 0.6)
test_size =int(len(idx) * 0.8)

data_all.iloc[idx[:train_size], :].to_csv('data/task2_train.csv',index=False)
data_all.iloc[idx[train_size:test_size], :].to_csv("data/task2_test.csv", index=False)
data_all.iloc[idx[test_size:], :].to_csv("data/task2_dev.csv", index=False)


In [5]:
#Torchtext采用声明式方法加载数据
from torchtext import data
PAD_TOKEN='<pad>'
TEXT = data.Field(sequential=True,batch_first=True, lower=True, pad_token=PAD_TOKEN)
LABEL = data.Field(sequential=False, batch_first=True, unk_token=None)


In [6]:
#读取数据

datafields = [("PhraseId", None), # 不需要的filed设置为None
              ("SentenceId", None),
              ('Phrase', TEXT),
              ('Sentiment', LABEL)]
train_data = data.TabularDataset(path='data/task2_train.csv', format='csv',
                                fields=datafields)
dev_data  = data.TabularDataset(path='data/task2_dev.csv', format='csv',
                                fields=datafields)
test_data = data.TabularDataset(path='data/task2_test.csv', format='csv',
                                fields=datafields)


In [7]:
print(len(train_data))

93637


In [8]:
#构建词典，字符映射到embedding
TEXT.build_vocab(train_data,  vectors= 'glove.6B.50d',   #可以提前下载好
                 unk_init= lambda x:torch.nn.init.uniform_(x, a=-0.25, b=0.25))
LABEL.build_vocab(train_data)

#得到索引，PAD_TOKEN='<pad>'
PAD_INDEX = TEXT.vocab.stoi[PAD_TOKEN]
TEXT.vocab.vectors[PAD_INDEX] = 0.0

In [9]:
#得到词向量
#pretrained_embeddings=TEXT.vocab.vectors
#print(type(TEXT.vocab.vectors))
#print(TEXT.vocab.vectors)

In [10]:
#构建迭代器
train_iterator = data.BucketIterator(train_data, batch_size=BATCH_SIZE, 
                                     train=True, shuffle=True,device=DEVICE)

dev_iterator = data.Iterator(dev_data, batch_size=len(dev_data), train=False,
                         sort=False, device=DEVICE)

test_iterator = data.Iterator(test_data, batch_size=len(test_data), train=False,
                          sort=False, device=DEVICE)


In [11]:
#部分参数设置
embedding_choice='glove'   #  'static'    'non-static'
num_embeddings = len(TEXT.vocab)
embedding_dim =50
dropout_p=0.5
filters_num=100

In [12]:
vocab_size=len(TEXT.vocab)
label_num=len(LABEL.vocab)
print(vocab_size,label_num)

16473 6


In [13]:
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.embedding_choice=embedding_choice
        self.vocab_size=vocab_size
        
        if self.embedding_choice==  'rand':
            self.embedding=nn.Embedding(num_embeddings,embedding_dim)
        if self.embedding_choice==  'glove':
            self.embedding = nn.Embedding(num_embeddings, embedding_dim, 
                padding_idx=PAD_INDEX).from_pretrained(TEXT.vocab.vectors, freeze=True)
            
            
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=filters_num ,  #卷积产生的通道
                               kernel_size=(3, embedding_dim), padding=(2,0))
        
        self.conv2 = nn.Conv2d(in_channels=1,out_channels=filters_num ,  #卷积产生的通道
                               kernel_size=(4, embedding_dim), padding=(3,0))
        
        self.conv3 = nn.Conv2d(in_channels=1,out_channels=filters_num ,  #卷积产生的通道
                               kernel_size=(5, embedding_dim), padding=(4,0))
        
        self.dropout = nn.Dropout(dropout_p)
        
        self.fc = nn.Linear(filters_num * 3, label_num)
        
    def forward(self,x):      # (Batch_size, Length) 
        x=self.embedding(x).unsqueeze(1)      #(Batch_size, Length, Dimention) 
                                       #(Batch_size, 1, Length, Dimention) 
        
        x1 = F.relu(self.conv1(x)).squeeze(3)    #(Batch_size, filters_num, length+padding, 1) 
                                          #(Batch_size, filters_num, length+padding) 
        x1 = F.max_pool1d(x1, x1.size(2)).squeeze(2)  #(Batch_size, filters_num, 1)
                                               #(Batch_size, filters_num) 
         
        x2 = F.relu(self.conv2(x)).squeeze(3)  
        x2 = F.max_pool1d(x2, x2.size(2)).squeeze(2)      
        
        x3 = F.relu(self.conv3(x)).squeeze(3)  
        x3 = F.max_pool1d(x3, x3.size(2)).squeeze(2)      
        
        x = torch.cat((x1, x2, x3), dim=1)  #(Batch_size, filters_num *3 )
        x = self.dropout(x)      #(Batch_size, filters_num *3 )
        out = self.fc(x)       #(Batch_size, label_num  )
        return out
    

In [14]:
#构建模型

model=CNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)#创建优化器SGD
criterion = nn.CrossEntropyLoss()   #损失函数

if USE_CUDA:
    model.cuda()

In [ ]:
import time
epoch=100
best_accuracy=0.0
start_time=time.time()

for i in range(epoch):
    model.train()
    total_loss=0.0
    accuracy=0.0
    total_correct=0.0
    total_data_num = len(train_iterator.dataset)
    steps = 0.0
    #训练
    for batch in train_iterator:
        steps+=1
        #print(steps)
        optimizer.zero_grad() #  梯度缓存清零
        
        batch_text=batch.Phrase
        batch_label=batch.Sentiment
        out=model(batch_text)    #[batch_size, label_num]
        loss = criterion(out, batch_label)
        total_loss = total_loss + loss.item() 

        loss.backward()
        optimizer.step()        

        correct = (torch.max(out, dim=1)[1]  #get the indices
                   .view(batch_label.size()) == batch_label).sum()
        total_correct = total_correct + correct.item()

        if steps%100==0:
            print("Epoch %d_%.3f%%:  Training average Loss: %f, Training accuracy: %f"
                      %(i, steps * train_iterator.batch_size*100/len(train_iterator.dataset),total_loss/steps, total_correct/total_data_num))  

    #验证
    model.eval()
    total_loss=0.0
    accuracy=0.0
    total_correct=0.0
    total_data_num = len(train_iterator.dataset)
    steps = 0.0    
    for batch in dev_iterator:
        steps+=1
        batch_text=batch.Phrase
        batch_label=batch.Sentiment
        out=model(batch_text)
        loss = criterion(out, batch_label)
        total_loss = total_loss + loss.item()
        
        correct = (torch.max(out, dim=1)[1].view(batch_label.size()) == batch_label).sum()
        total_correct = total_correct + correct.item()
        
        print("Epoch %d :  Verification average Loss: %f, Verification accuracy: %f%%,Total Time:%f"
          %(i, total_loss/steps, total_correct*100/total_data_num,time.time()-start_time))  
        
        if best_accuracy < total_correct/total_data_num :
            best_accuracy =total_correct/total_data_num 
            torch.save(model,'model_dict/model_glove/epoch_%d_accuracy_%f'%(i,total_correct/total_data_num))
            print('Model is saved in model_dict/model_glove/epoch_%d_accuracy_%f'%(i,total_correct/total_data_num))
            #torch.cuda.empty_cache()

Epoch 0_1.068%:  Training average Loss: 1.378938, Training accuracy: 0.005190
Epoch 0_2.136%:  Training average Loss: 1.312844, Training accuracy: 0.010754
Epoch 0_3.204%:  Training average Loss: 1.294743, Training accuracy: 0.016180
Epoch 0_4.272%:  Training average Loss: 1.271463, Training accuracy: 0.021744
Epoch 0_5.340%:  Training average Loss: 1.247341, Training accuracy: 0.027628
Epoch 0_6.408%:  Training average Loss: 1.219641, Training accuracy: 0.033790
Epoch 0_7.476%:  Training average Loss: 1.201916, Training accuracy: 0.039653
Epoch 0_8.544%:  Training average Loss: 1.186622, Training accuracy: 0.045751
Epoch 0_9.612%:  Training average Loss: 1.174941, Training accuracy: 0.051710
Epoch 0_10.680%:  Training average Loss: 1.165636, Training accuracy: 0.057648
Epoch 0_11.747%:  Training average Loss: 1.155986, Training accuracy: 0.063842
Epoch 0_12.815%:  Training average Loss: 1.152129, Training accuracy: 0.069609
Epoch 0_13.883%:  Training average Loss: 1.148105, Training a

C:\Users\htfhxx\.conda\envs\python36\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model is saved in model_dict/model_glove/epoch_0_accuracy_0.199996
Epoch 1_1.068%:  Training average Loss: 1.034252, Training accuracy: 0.006301
Epoch 1_2.136%:  Training average Loss: 1.037600, Training accuracy: 0.012399
Epoch 1_3.204%:  Training average Loss: 1.025903, Training accuracy: 0.018582
Epoch 1_4.272%:  Training average Loss: 1.032115, Training accuracy: 0.024680
Epoch 1_5.340%:  Training average Loss: 1.027209, Training accuracy: 0.031077
Epoch 1_6.408%:  Training average Loss: 1.029459, Training accuracy: 0.037101
Epoch 1_7.476%:  Training average Loss: 1.030277, Training accuracy: 0.043338
Epoch 1_8.544%:  Training average Loss: 1.026520, Training accuracy: 0.049660
Epoch 1_9.612%:  Training average Loss: 1.027233, Training accuracy: 0.055865
Epoch 1_10.680%:  Training average Loss: 1.025347, Training accuracy: 0.062166
Epoch 1_11.747%:  Training average Loss: 1.021690, Training accuracy: 0.068595
Epoch 1_12.815%:  Training average Loss: 1.021992, Training accuracy: 0.0

Epoch 2_8.544%:  Training average Loss: 0.992123, Training accuracy: 0.050877
Epoch 2_9.612%:  Training average Loss: 0.994701, Training accuracy: 0.057125
Epoch 2_10.680%:  Training average Loss: 0.991011, Training accuracy: 0.063757
Epoch 2_11.747%:  Training average Loss: 0.992143, Training accuracy: 0.070004
Epoch 2_12.815%:  Training average Loss: 0.995540, Training accuracy: 0.076508
Epoch 2_13.883%:  Training average Loss: 0.997782, Training accuracy: 0.082724
Epoch 2_14.951%:  Training average Loss: 1.000065, Training accuracy: 0.089046
Epoch 2_16.019%:  Training average Loss: 0.999585, Training accuracy: 0.095336
Epoch 2_17.087%:  Training average Loss: 0.997646, Training accuracy: 0.101733
Epoch 2_18.155%:  Training average Loss: 0.995088, Training accuracy: 0.108162
Epoch 2_19.223%:  Training average Loss: 0.995357, Training accuracy: 0.114389
Epoch 2_20.291%:  Training average Loss: 0.993864, Training accuracy: 0.120807
Epoch 2_21.359%:  Training average Loss: 0.995542, Tra

Epoch 3_17.087%:  Training average Loss: 0.983085, Training accuracy: 0.102513
Epoch 3_18.155%:  Training average Loss: 0.982515, Training accuracy: 0.108921
Epoch 3_19.223%:  Training average Loss: 0.979945, Training accuracy: 0.115467
Epoch 3_20.291%:  Training average Loss: 0.979367, Training accuracy: 0.121843
Epoch 3_21.359%:  Training average Loss: 0.979073, Training accuracy: 0.128496
Epoch 3_22.427%:  Training average Loss: 0.978709, Training accuracy: 0.134915
Epoch 3_23.495%:  Training average Loss: 0.978266, Training accuracy: 0.141493
Epoch 3_24.563%:  Training average Loss: 0.978802, Training accuracy: 0.147997
Epoch 3_25.631%:  Training average Loss: 0.979307, Training accuracy: 0.154362
Epoch 3_26.699%:  Training average Loss: 0.980158, Training accuracy: 0.160716
Epoch 3_27.767%:  Training average Loss: 0.979680, Training accuracy: 0.167092
Epoch 3_28.835%:  Training average Loss: 0.979546, Training accuracy: 0.173478
Epoch 3_29.903%:  Training average Loss: 0.978495, T

...

Epoch 26 :  Verification average Loss: 0.924410, Verification accuracy: 21.016265%,Total Time:4344.645683
Epoch 27_1.068%:  Training average Loss: 0.813154, Training accuracy: 0.007198
Epoch 27_2.136%:  Training average Loss: 0.846484, Training accuracy: 0.013905
Epoch 27_3.204%:  Training average Loss: 0.825709, Training accuracy: 0.021124
Epoch 27_4.272%:  Training average Loss: 0.826494, Training accuracy: 0.028173
Epoch 27_5.340%:  Training average Loss: 0.833812, Training accuracy: 0.034922
Epoch 27_6.408%:  Training average Loss: 0.840689, Training accuracy: 0.041800
Epoch 27_7.476%:  Training average Loss: 0.840003, Training accuracy: 0.048816
Epoch 27_8.544%:  Training average Loss: 0.840772, Training accuracy: 0.055769
Epoch 27_9.612%:  Training average Loss: 0.847699, Training accuracy: 0.062657
Epoch 27_10.680%:  Training average Loss: 0.848609, Training accuracy: 0.069759
Epoch 27_11.747%:  Training average Loss: 0.849658, Training accuracy: 0.076636
Epoch 27_12.815%:  Trai

Epoch 28_7.476%:  Training average Loss: 0.850343, Training accuracy: 0.048485
Epoch 28_8.544%:  Training average Loss: 0.854132, Training accuracy: 0.055299
Epoch 28_9.612%:  Training average Loss: 0.854089, Training accuracy: 0.062230
Epoch 28_10.680%:  Training average Loss: 0.850672, Training accuracy: 0.069396
Epoch 28_11.747%:  Training average Loss: 0.854136, Training accuracy: 0.076241
Epoch 28_12.815%:  Training average Loss: 0.851531, Training accuracy: 0.083332
Epoch 28_13.883%:  Training average Loss: 0.850470, Training accuracy: 0.090520
Epoch 28_14.951%:  Training average Loss: 0.852474, Training accuracy: 0.097429
Epoch 28_16.019%:  Training average Loss: 0.852488, Training accuracy: 0.104350
Epoch 28_17.087%:  Training average Loss: 0.851068, Training accuracy: 0.111548
Epoch 28_18.155%:  Training average Loss: 0.852989, Training accuracy: 0.118276
Epoch 28_19.223%:  Training average Loss: 0.851034, Training accuracy: 0.125527
Epoch 28_20.291%:  Training average Loss: 0

Epoch 29_16.019%:  Training average Loss: 0.851866, Training accuracy: 0.105450
Epoch 29_17.087%:  Training average Loss: 0.853766, Training accuracy: 0.112253
Epoch 29_18.155%:  Training average Loss: 0.853658, Training accuracy: 0.119237
Epoch 29_19.223%:  Training average Loss: 0.854928, Training accuracy: 0.126093
Epoch 29_20.291%:  Training average Loss: 0.853965, Training accuracy: 0.133014
Epoch 29_21.359%:  Training average Loss: 0.853023, Training accuracy: 0.140105
Epoch 29_22.427%:  Training average Loss: 0.850543, Training accuracy: 0.147356
Epoch 29_23.495%:  Training average Loss: 0.853249, Training accuracy: 0.154020
Epoch 29_24.563%:  Training average Loss: 0.855219, Training accuracy: 0.160674
Epoch 29_25.631%:  Training average Loss: 0.856962, Training accuracy: 0.167477
Epoch 29_26.699%:  Training average Loss: 0.855412, Training accuracy: 0.174664
Epoch 29_27.767%:  Training average Loss: 0.856757, Training accuracy: 0.181531
Epoch 29_28.835%:  Training average Loss

Epoch 30_24.563%:  Training average Loss: 0.853772, Training accuracy: 0.160033
Epoch 30_25.631%:  Training average Loss: 0.853146, Training accuracy: 0.167060
Epoch 30_26.699%:  Training average Loss: 0.851855, Training accuracy: 0.174076
Epoch 30_27.767%:  Training average Loss: 0.851254, Training accuracy: 0.181242
Epoch 30_28.835%:  Training average Loss: 0.850321, Training accuracy: 0.188323
Epoch 30_29.903%:  Training average Loss: 0.850780, Training accuracy: 0.195169
Epoch 30_30.971%:  Training average Loss: 0.851110, Training accuracy: 0.201971
Epoch 30_32.039%:  Training average Loss: 0.851209, Training accuracy: 0.208860
Epoch 30_33.107%:  Training average Loss: 0.852945, Training accuracy: 0.215631
Epoch 30_34.175%:  Training average Loss: 0.854277, Training accuracy: 0.222177
Epoch 30_35.242%:  Training average Loss: 0.852913, Training accuracy: 0.229258
Epoch 30_36.310%:  Training average Loss: 0.853618, Training accuracy: 0.236146
Epoch 30_37.378%:  Training average Loss

Epoch 31_32.039%:  Training average Loss: 0.849883, Training accuracy: 0.209629
Epoch 31_33.107%:  Training average Loss: 0.850843, Training accuracy: 0.216592
Epoch 31_34.175%:  Training average Loss: 0.852001, Training accuracy: 0.223523


In [18]:
#测试
PATH='model_dict/model_glove/epoch_6_accuracy_0.213879'
model = torch.load(PATH)

total_loss=0.0
accuracy=0.0
total_correct=0.0
total_data_num = len(train_iterator.dataset)
steps = 0.0    
start_time=time.time()
for batch in test_iterator:
    steps+=1
    batch_text=batch.Phrase
    batch_label=batch.Sentiment
    out=model(batch_text)
    loss = criterion(out, batch_label)
    total_loss = total_loss + loss.item()

    correct = (torch.max(out, dim=1)[1].view(batch_label.size()) == batch_label).sum()
    total_correct = total_correct + correct.item()

print("Test average Loss: %f, Test accuracy: %f，Total time: %f"
  %(total_loss/steps, total_correct/total_data_num,time.time()-start_time) ) 

Test average Loss: 0.942344, Test accuracy: 0.212843，Total time: 16.872462
